In [ ]:
import pandas as pd
import numpy as np

import math

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

In [ ]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'project-donny-347713-df9944bf12d3.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)


In [ ]:
file = gc.open("Raw Parkiran PT. SUMO UTOMO")

name_sheet = ['Jan-2021', 'Feb-2021', 'Mar-2021', 'Apr-2021', 'Mei-2021', 'Jun-2021', 'Jul-2021',
              'Agu-2021', 'Sep-2021', 'Okt-2021', 'Nov-2021', 'Des-2021']

data = pd.DataFrame(columns=['Gate', 'Vehicle', 'Entry', 'Exit'])

for i in name_sheet:
  sheet = file.worksheet(i)
  df = pd.DataFrame(sheet.get_all_records())
  data = data.append(df)

In [ ]:
#mengubah datatype object ke datetime
data['Entry'] = pd.to_datetime(data['Entry'])
data['Exit'] = pd.to_datetime(data['Exit'])
data['Month'] = data['Entry'].dt.month_name()
data['Validasi'] = data['Exit'] > data['Entry']

data.head(3)

,Gate,Vehicle,Entry,Exit,Month,Validasi
0,B2,Motorcycle,2021-01-06 22:23:57,2021-01-19 23:45:02,January,True
1,B3,Car,2021-01-07 05:43:32,2021-01-06 04:36:17,January,False
2,A1,Car,2021-01-06 04:47:47,2021-01-24 00:58:50,January,True


In [ ]:
# 1. Berapa besar proporsi error secara keseluruhan dan per bulan

In [ ]:
report_1 = data.groupby(['Month', 'Validasi']).agg({'Entry':'count'}).unstack().reset_index()
report_1.columns = ['Month', 'FALSE', 'TRUE']
report_1['PERSENTASE (%)'] = (report_1['FALSE'] / (report_1['TRUE'] + report_1['FALSE']))*100
report_1.to_excel('Report_1.xlsx', index=False)
report_1

,Month,FALSE,TRUE,PERSENTASE (%)
0,April,53,47,53.0
1,August,54,46,54.0
2,December,57,43,57.0
3,February,46,54,46.0
4,January,48,52,48.0
5,July,55,45,55.0
6,June,54,46,54.0
7,March,46,54,46.0
8,May,41,59,41.0
9,November,53,47,53.0


In [ ]:
# 2. Berapa pendapatannya? ketentuan parkir mobil 3000/jam, parkir motor 2000/jam

In [ ]:
result = data[data['Validasi'] == True].reset_index()
result

In [ ]:
def get_hours(start, end):
  td = (start - end)
  min = td.total_seconds()/60
  hr = min / 60
  days = hr / 24
  
  return math.ceil(hr)

for index, row in result.iterrows():
  result.loc[index, 'Duration'] = get_hours(row['Exit'], row['Entry'])

result.loc[result['Vehicle'] == 'Car', 'Price'] = result['Duration']*3000
result.loc[result['Vehicle'] == 'Motorcycle', 'Price'] = result['Duration']*2000

In [ ]:
report_2 = result.groupby(['Month']).agg({'Price':'sum'}).reset_index()
report_2.to_excel('Report_2.xlsx', index=False)
report_2

,Month,Price
0,April,30853000.0
1,August,30605000.0
2,December,30419000.0
3,February,29484000.0
4,January,29871000.0
5,July,19737000.0
6,June,23418000.0
7,March,31814000.0
8,May,41624000.0
9,November,32319000.0


In [ ]:
# 3. Pendapatan tiap bulan berdasarkan jenis kendaraan 

In [ ]:
report_3 = result.groupby(['Month', 'Vehicle']).agg({'Price':'sum'}).reset_index()
report_3.to_excel('Report_2.xlsx', index=False)
report_3

,Month,Vehicle,Price
0,April,Car,18351000.0
1,April,Motorcycle,12502000.0
2,August,Car,14367000.0
3,August,Motorcycle,16238000.0
4,December,Car,23067000.0
5,December,Motorcycle,7352000.0
6,February,Car,13974000.0
7,February,Motorcycle,15510000.0
8,January,Car,13065000.0
9,January,Motorcycle,16806000.0


In [2]:
import pandas as pd

city_id = [100, 101, 102, 103, 104, 105, 106]
city_name = ['Jakarta', 'Bandung', 'Surabaya', 'Pontianak', 'Medan', 'Makassar', 'Jayapura']
df = pd.DataFrame(list(zip(city_id, city_name)), columns=['ID', 'City'])
df

,ID,City
0,100,Jakarta
1,101,Bandung
2,102,Surabaya
3,103,Pontianak
4,104,Medan
5,105,Makassar
6,106,Jayapura


In [3]:
encoded = pd.get_dummies(df, prefix='City')
encoded

,ID,City_Bandung,City_Jakarta,City_Jayapura,City_Makassar,City_Medan,City_Pontianak,City_Surabaya
0,100,0,1,0,0,0,0,0
1,101,1,0,0,0,0,0,0
2,102,0,0,0,0,0,0,1
3,103,0,0,0,0,0,1,0
4,104,0,0,0,0,1,0,0
5,105,0,0,0,1,0,0,0
6,106,0,0,1,0,0,0,0


In [15]:
#importing the libraries
import pandas as pd
from sklearn import preprocessing

country = ['India', 'USA', 'Japan', 'USA', 'Japan']
age = [45,35,47,36,24]
salary = [72000, 65000, 88000, 45000, 36000]
df = pd.DataFrame(list(zip(country, age, salary)), columns=['Country', 'Age', 'Salary'])
df

,Country,Age,Salary
0,India,45,72000
1,USA,35,65000
2,Japan,47,88000
3,USA,36,45000
4,Japan,24,36000


In [16]:
label_encoder = preprocessing.LabelEncoder()
df['Country']= label_encoder.fit_transform(df['Country']) 
df.head()

,Country,Age,Salary
0,0,45,72000
1,2,35,65000
2,1,47,88000
3,2,36,45000
4,1,24,36000
